# Lesson 7: RAG & Knowledge Bases

🟡 **Intermediate** · ⏱ **30 min**

---

## Overview

**RAG (Retrieval-Augmented Generation)** grounds your agent's answers in your own documents. Instead of relying on the LLM's training data, the agent first searches your knowledge base for relevant chunks, then uses them to answer.

Result: **accurate, up-to-date responses from your data.**

---

## What You'll Learn

By the end of this lesson you will be able to:

1. **Create** a persistent cloud knowledge base
2. **Add** text and website content to it
3. **Query** the knowledge base directly to inspect retrieved chunks
4. **Attach** a knowledge base to an agent for automatic RAG
5. **Manage** documents — list, delete, reset

## Prerequisites

Before running this notebook, make sure you have completed:

| Lesson | Topic |
|--------|-------|
| Lesson 1 | Getting Started — `Studio` and `agent.run()` |
| Lesson 2 | Providers and Models |
| Lesson 3 | Agent Lifecycle |
| Lesson 4 | Structured Outputs |
| Lesson 5 | Memory and Sessions |
| Lesson 6 | Tools and Functions |

You also need a valid **`LYZR_API_KEY`** set as an environment variable (or paste it directly in the setup cell below).

In [ ]:
!pip install lyzr-adk -q

In [ ]:
import os
from lyzr import Studio

API_KEY = os.getenv("LYZR_API_KEY", "YOUR_LYZR_API_KEY")
studio = Studio(api_key=API_KEY)
print("Ready!")

## Concept 1: What is RAG?

Without RAG, an LLM answers from its training data — which may be **outdated, incomplete, or simply wrong** for your domain.

With RAG, the agent first **retrieves relevant chunks** from your documents, then passes them alongside the user's question to the LLM. The LLM grounds its answer in your actual content.

```
User question
     |
     v
[Query KB] --> retrieve top-k relevant chunks
     |
     v
[LLM] <-- question + chunks
     |
     v
Grounded answer
```

**Without RAG:** the LLM guesses based on training data.

**With RAG:** the LLM answers from your actual documents — accurate, citable, up-to-date.

### Key Terms

| Term | Meaning |
|------|---------|
| **Chunk** | A small piece of text that gets embedded and stored in the vector store |
| **Embedding** | A numerical representation of text, used for similarity search |
| **top_k** | Number of most-relevant chunks to retrieve |
| **score_threshold** | Minimum similarity score for a chunk to be returned |

## Concept 2: Creating a Knowledge Base

A **knowledge base** is a persistent cloud resource — think of it as a managed vector store. You create it once, populate it with content, and reuse it across multiple agents.

Supported vector stores: **Qdrant, Weaviate, PG-Vector, Milvus, Neptune**.

```python
kb = studio.create_knowledge_base(name="My KB")
```

This returns a `KnowledgeBase` object with an `.id` you can use to retrieve it later.

In [ ]:
# Create a knowledge base
kb = studio.create_knowledge_base(name="Lyzr Product FAQ")
print(f"Knowledge base created!")
print(f"  Name: {kb.name}")
print(f"  ID:   {kb.id}")

## Concept 3: Adding Text Content

The simplest way to add content is `kb.add_text()`. Pass a string and a **source label**.

Source labels let you:
- **Trace** which document a chunk came from
- **Delete** specific documents later by ID
- **Audit** your knowledge base contents

```python
kb.add_text(text="...", source="descriptive-label")
```

Use descriptive, versioned labels like `product-faq-v1` or `policy-2025-01` for easy management.

In [ ]:
faq_content = """
# Lyzr ADK FAQ

## What is Lyzr ADK?
Lyzr ADK (Agent Development Kit) is a Python SDK for building production-ready AI agents.
It supports 20+ LLM models across providers like OpenAI, Anthropic, Google, and Groq.

## What LLM providers are supported?
Lyzr ADK supports: OpenAI (GPT-4o, o3), Anthropic (Claude Sonnet, Opus),
Google (Gemini 2.0, 2.5), Groq (Llama 3.3), Perplexity (Sonar Pro),
and AWS Bedrock (Nova, Claude, Llama).

## Is there a free tier?
Yes! Sign up at lyzr.ai to get started with free credits.
No credit card required for the free tier.

## Can I add memory to my agents?
Yes. Call agent.add_memory(max_messages=10) to enable conversation memory.
Supported providers: Lyzr (built-in), Mem0, AWS AgentCore, SuperMemory.

## What vector stores are supported for knowledge bases?
Lyzr supports: Qdrant, Weaviate, PG-Vector, Milvus, and Neptune.
"""

kb.add_text(text=faq_content, source="product-faq-v1")
print("FAQ content added!")

## Concept 4: Adding Website Content

Use `kb.add_website()` to crawl and index a URL. Lyzr fetches the pages, extracts text, and indexes them as chunks.

| Parameter | Description | Recommendation |
|-----------|-------------|----------------|
| `url` | Starting URL to crawl | Use the root docs page |
| `max_pages` | Maximum number of pages to fetch | Start small (3-10) |
| `max_depth` | How deep to follow links from the starting URL | `1` = only the given page, `2` = one level of links |

```python
kb.add_website(url="https://docs.lyzr.ai", max_pages=5, max_depth=1)
```

> Note: Crawling and indexing happens asynchronously. Large sites may take a minute before content is searchable.

In [ ]:
# Add documentation from a website
# max_pages: max number of pages to crawl
# max_depth: how deep to follow links (1 = only the given page)
kb.add_website(
    url="https://docs.lyzr.ai",
    max_pages=5,
    max_depth=1
)
print("Website content added!")
print("(Note: indexing may take a moment)")

## Concept 5: Querying the Knowledge Base Directly

Before attaching a KB to an agent, **test what gets retrieved**. This lets you tune your content and parameters without burning agent inference tokens.

```python
results = kb.query(query="...", top_k=3, score_threshold=0.0)
for r in results:
    print(r.score, r.text)
```

**Tuning tips:**

- **`top_k`**: Higher = more context passed to the LLM, but also more noise. Start with 3-5.
- **`score_threshold`**: Set to `0.5`-`0.7` to filter out weakly-related chunks. Use `0.0` to see all results.
- If results look wrong, check that your **source content is clear and specific**.

In [ ]:
# Query the knowledge base
results = kb.query(
    query="What LLM providers does Lyzr support?",
    top_k=3,
    score_threshold=0.0   # 0.0 = return all results regardless of score
)

print(f"Found {len(results)} results:\n")
for i, result in enumerate(results, 1):
    print(f"Result {i} (score: {result.score:.3f}):")
    print(f"  {result.text[:200]}...")
    print()

## Concept 6: Attaching a Knowledge Base to an Agent

Pass `knowledge_base_ids=[kb.id]` when creating an agent. From that point on, **every call to `agent.run()`** automatically:

1. Embeds the user question
2. Retrieves the most relevant chunks from the KB
3. Injects those chunks into the LLM prompt as context
4. Returns a grounded answer

You can attach **multiple knowledge bases** to a single agent:

```python
agent = studio.create_agent(
    ...,
    knowledge_base_ids=[kb1.id, kb2.id]
)
```

The instructions should guide the agent on how to handle missing information (e.g., say "I don't have that information" rather than hallucinating).

In [ ]:
# Create an agent that uses the knowledge base
faq_agent = studio.create_agent(
    name="Lyzr FAQ Bot",
    provider="openai/gpt-4o",
    role="Product support specialist",
    goal="Answer questions about Lyzr ADK accurately using the knowledge base",
    instructions=(
        "Answer only using information from the provided knowledge base. "
        "If the answer isn't in the knowledge base, say 'I don't have that information.' "
        "Be concise and direct."
    ),
    knowledge_base_ids=[kb.id]
)
print(f"Agent created with KB: {faq_agent.id}")

# Test it
r1 = faq_agent.run("What LLM providers are supported?")
print(f"\nQ: What LLM providers are supported?")
print(f"A: {r1.response}")

r2 = faq_agent.run("Is there a free tier?")
print(f"\nQ: Is there a free tier?")
print(f"A: {r2.response}")

## Concept 7: Managing Documents

The `KnowledgeBase` object provides full CRUD control over your content:

| Method | Description |
|--------|-------------|
| `kb.list_documents()` | Returns all documents with IDs and metadata |
| `kb.delete_documents([id1, id2])` | Remove specific documents by ID |
| `kb.reset()` | Remove **all** documents but keep the KB |
| `kb.delete()` | Delete the KB entirely (permanent) |

Use `kb.reset()` when you want to reload fresh content without deleting the KB (and recreating agents that reference its ID).

In [ ]:
# List all documents in the KB
docs = kb.list_documents()
print(f"Documents in knowledge base: {len(docs)}")
for doc in docs:
    print(f"  * {doc.id} -- source: {getattr(doc, 'source', 'unknown')}")

# Delete a specific document
if docs:
    first_doc_id = docs[0].id
    kb.delete_documents([first_doc_id])
    print(f"\nDeleted document: {first_doc_id}")
    print(f"Remaining documents: {len(kb.list_documents())}")

## Common Mistake: Querying Before Indexing Completes

**Problem:** You call `kb.add_website()` or `kb.add_text()` with large content and immediately query — and get 0 results.

**Why:** Indexing (chunking + embedding + storing) is asynchronous. The content isn't searchable until it's fully processed.

**Fix:**
- For small text: a 1-2 second pause is usually enough
- For websites or large documents: wait longer, or check `kb.list_documents()` until the count stabilizes
- In production: use a webhook or polling loop rather than a fixed sleep

```python
# Wrong
kb.add_website(url="...", max_pages=10)
results = kb.query("question")  # May return 0 results -- not indexed yet!

# Better
import time
kb.add_text("...", source="...")
time.sleep(2)  # Brief pause for small content
results = kb.query("question")
```

In [ ]:
# Wrong: querying immediately after add_website() before indexing completes
# kb.add_website(url="...", max_pages=10)
# results = kb.query("question")  # May return 0 results -- not indexed yet!

# Better: add a short wait or check document count
import time

fresh_kb = studio.create_knowledge_base(name="Quick Test KB")
fresh_kb.add_text("The capital of France is Paris.", source="geography")

# Small text usually indexes quickly
time.sleep(2)  # Brief pause for indexing

test_results = fresh_kb.query("What is the capital of France?", top_k=1)
print(f"Results after indexing: {len(test_results)}")
if test_results:
    print(f"Found: {test_results[0].text}")

fresh_kb.delete()  # Clean up
print("Test KB deleted.")

## Exercise

Build a knowledge base on **a topic you know well** — your company's product, a technical domain, a hobby, or anything else.

### Steps

1. Create a knowledge base with a meaningful name
2. Add **at least 3 text sections** with different source labels
3. Query it with **3 different questions** and print the top result for each
4. Create an agent with the KB attached and test it with a question

### Stretch Goals

- Add a website with `kb.add_website()` in addition to text
- Try different `score_threshold` values (0.0, 0.3, 0.7) and compare results
- Delete one document and verify the agent no longer answers questions about it

In [ ]:
# TODO: Create a knowledge base on a topic you know well
my_kb = studio.create_knowledge_base(name=...)

# TODO: Add at least 3 text sections with different sources
my_kb.add_text(text=..., source=...)
my_kb.add_text(text=..., source=...)
my_kb.add_text(text=..., source=...)

# TODO: Query it with 3 questions and print the top result for each
for question in [..., ..., ...]:
    results = my_kb.query(question, top_k=1)
    if results:
        print(f"Q: {question}")
        print(f"A: {results[0].text[:150]}...\n")

# TODO: Create an agent with the KB and test it
my_agent = studio.create_agent(
    name=...,
    provider="openai/gpt-4o",
    role=...,
    goal=...,
    instructions=...,
    knowledge_base_ids=[my_kb.id]
)

response = my_agent.run(...)
print(response.response)

## Summary

### Knowledge Base API

| Method | Description |
|--------|-------------|
| `studio.create_knowledge_base(name=...)` | Create a new KB |
| `studio.get_knowledge_base(kb_id)` | Retrieve an existing KB by ID |
| `studio.list_knowledge_bases()` | List all your KBs |
| `kb.add_text(text, source)` | Index a text string |
| `kb.add_pdf(file_path, chunk_size, chunk_overlap)` | Index a PDF file |
| `kb.add_website(url, max_pages, max_depth)` | Crawl and index a website |
| `kb.query(query, top_k, score_threshold)` | Retrieve relevant chunks |
| `kb.list_documents()` | List all documents |
| `kb.delete_documents([id])` | Delete specific documents |
| `kb.reset()` | Remove all documents, keep the KB |
| `kb.delete()` | Delete the KB entirely |

### RAG Pipeline (Recap)

```
User question
     |
     v
[Query KB] --> retrieve top-k relevant chunks
     |
     v
[LLM] <-- question + chunks
     |
     v
Grounded answer
```

### Key Takeaways

- A knowledge base is a **persistent cloud resource** — create once, reuse across agents
- Use **descriptive source labels** for traceability and selective deletion
- **Query directly** before attaching to an agent to validate retrieved chunks
- Pass `knowledge_base_ids=[kb.id]` to an agent for automatic RAG on every `run()` call
- **Indexing is asynchronous** — always allow time before querying newly added content
- Write instructions that tell the agent what to do when the KB has no answer (prevents hallucination)

## Next Steps

**Lesson 8: Dynamic Contexts** — Learn how to inject runtime data into your agent's context window: user profiles, live API results, session state, and more.

---

| Lesson | Topic | Status |
|--------|-------|--------|
| 1 | Getting Started | Done |
| 2 | Providers and Models | Done |
| 3 | Agent Lifecycle | Done |
| 4 | Structured Outputs | Done |
| 5 | Memory and Sessions | Done |
| 6 | Tools and Functions | Done |
| **7** | **RAG and Knowledge Bases** | **You are here** |
| 8 | Dynamic Contexts | Up next |
| 9 | Multi-Agent Workflows | Coming soon |
| 10 | Production Deployment | Coming soon |